# 02 - Portfolios

In this notebook you can find simple examples that showcase how to programatically create portfolios.

In [1]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2
%pip install seaborn

You should consider upgrading via the '/home/saheru/Documents/Programming/Finance/IBKR/ibeam/env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Start server

The `Server` class is the interface to perform HTTPS requests to acquire data regarding instruments, like stocks.

In [2]:
import time
import logging

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from modules.server import Server
from modules.instruments import Portfolio

In [3]:
srv = Server()
srv.start()

# the following silences the logging output
logging.getLogger('ibkr-algotrading').setLevel(logging.ERROR)

Once started, we can request information on any stock we want simply by accesing items in `srv`. These will be cached, so only the first access will have significant delay.

First, it is a good idea to know which is our balance,

In [4]:
srv.balance

(1000000.0, 'USD')

> NOTE: with a simulated IBKR Paper Account you start with 1'000'000 USD, which you can use to test out trading.

## Create Portfolio

we can now request some stocks and create a portfolio with them,

In [5]:
stocks = srv[['SPY', 'XOM', 'MSFT', 'AAPL', 'TSLA'], '1y', '1d']

portfolio = Portfolio(wealth=100, stocks=stocks)

The `wealth` argument is in the same currency as our account's balance.

We can now check how much fraction of stock is allocated for each of the stocks.

Of course, right now we haven't bought any stock so it's all zero,

In [6]:
portfolio.alloc

array([0., 0., 0., 0., 0.])

Similarly, our distribution of wealth accross the stocks looks like an all-zero array, and
this is the only time it will be like so before we perform our first order.

In any other case the "weights" will sum up to 1.

In [7]:
portfolio.weights

array([0., 0., 0., 0., 0.])

we finally add the portfolio to the server so it knows about its existance,

In [8]:
srv.add_portfolio(portfolio)

## Place an order on our portfolio

To prevent accidentally losing all of our money, we will trade under a simulated context in which orders are only previewed but we still get whole information on what would happen,

In [9]:
with srv.simulated():
    # 1st order
    # distribute the portfolio equally accross stocks
    weights = np.ones(5) / 5.0
    print('1st order')
    portfolio.order(weights)

    # the wealth of the portfolio decreases since the
    # transactions take a commission
    print('Portfolio wealth: ', portfolio.wealth)
    print('Total commissions:', portfolio.commission, 'Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # 2nd order
    # we will put all our money on SPY
    weights[0] = 1.0
    weights[1:] = 0.0
    print('2nd order')
    portfolio.order(weights)

    print('Portfolio wealth: ', portfolio.wealth)
    print('Total commissions:', portfolio.commission, 'Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # 3rd order
    # now we will give a litle to XOM
    weights[0] = 0.9
    weights[1] = 0.1
    print('3rd order')
    portfolio.order(weights)

    print('Portfolio wealth: ', portfolio.wealth)
    print('Total commissions:', portfolio.commission, 'Wealth + commission: ', portfolio.wealth + portfolio.commission)

    # we can also list all orders that have happened on this portfolio.
    # the stored values are the stock shares
    print('Portfolio orders:')
    print(portfolio.orders)

1st order
Portfolio wealth:  99.02361400000001
Total commissions: 0.9900990000000001 Wealth + commission:  100.01371300000001
2nd order
Portfolio wealth:  98.02612
Total commissions: 1.9705308119778602 Wealth + commission:  100.00098681197788
3rd order
Portfolio wealth:  97.063008
Total commissions: 2.9411293765293003 Wealth + commission:  100.0041373765293
Portfolio orders:
                               SPY     XOM    MSFT    AAPL    TSLA
2021-08-31 17:44:55.406606  0.0438  0.3600  0.0652  0.1304  0.0269
2021-08-31 17:44:58.684041  0.2168  0.0000  0.0000  0.0000  0.0000
2021-08-31 17:45:03.134897  0.1932  0.1764  0.0000  0.0000  0.0000


Notice how the portfolio's overall wealth decreases after each order since there are commissions.

Also note that after we exit the simulated context, the whole state of the server will be restored, meaning all orders in the
portfolio will disappear and it will return to its previous state.

In [10]:
# after exiting the simulation, the portfolio does not have any orders
print('Portfolio orders:')
print(portfolio.orders)

Portfolio orders:
Empty DataFrame
Columns: []
Index: []


## Place an order back-in-time and check statistics

In a simulated context we can also "place orders" in the past by using historical data. This allows us to easily study the performance of the portfolio. 

> NOTE: back-in-time orders only work in a simulated context.

In [11]:
# we will use as order date one year before today
first_date = pd.Timestamp.today() - pd.Timedelta(days=365)
print('1st date of order:', first_date)

with srv.simulated():
    # 1st order
    # distribute the portfolio equally accross stocks
    date = first_date
    weights = np.ones(5) / 5.0
    print('1st order')
    portfolio.order(weights, date=date)

    print('Today\'s portfolio wealth: ', portfolio.wealth)
    print('commission:', portfolio.commission, 'Today\'s wealth + commission: ', portfolio.wealth + portfolio.commission)

    # 2nd order
    # we will put all our money on SPY one month **after** the previous order
    date = date + pd.Timedelta(days=30)
    weights[0] = 1.0
    weights[1:] = 0.0
    print('2nd order')
    portfolio.order(weights, date=date)

    print('Today\'s portfolio wealth: ', portfolio.wealth)
    print('commission:', portfolio.commission, 'Today\'s wealth + commission: ', portfolio.wealth + portfolio.commission)

    # all orders that have happened on this portfolio.
    # the stored values are the stock shares
    print('Portfolio orders:')
    print(portfolio.orders)

    # we can also access the portfolio's statistics
    print('60 day Sharpe ratio:', portfolio.sharpe(end_date=first_date + pd.Timedelta(days=60)))
    print('Half year Sharpe ratio:', portfolio.sharpe(end_date=first_date + pd.Timedelta(days=365/2)))    

1st date of order: 2020-08-31 17:45:04.152281
1st order
Today's portfolio wealth:  132.12303599999998
commission: 0.9900990000000001 Today's wealth + commission:  133.12186899999998
2nd order
Today's portfolio wealth:  176.61760200000003
commission: 2.298334346665477 Today's wealth + commission:  178.91593634666552
Portfolio orders:
                               SPY     XOM    MSFT    AAPL    TSLA
2020-08-31 17:45:04.152281  0.0567  0.4958  0.0878  0.1535  0.0397
2020-09-30 17:45:04.152281  0.3906  0.0000  0.0000  0.0000  0.0000
60 day Sharpe ratio: 1.7059101202001883
Half year Sharpe ratio: 1.4952130965627688
